In [2]:
import numpy as np
import tflearn
from tflearn.layers.estimator import regression
import tensorflow as tf
import ipdb as pdb

In [3]:
path = 'data_set_LGD.npz'
res = np.load(path)
data_set = res['arr_0']
train_input = np.array([sample[0] for sample in data_set])
train_output = np.array([sample[1] for sample in data_set])

X = train_input[:350000,:]
Y = train_output[:350000,:]

testX = train_input[350000:,:]
testY = train_output[350000:,:]

In [9]:
Layer1 = 400
Layer2 = 300

inputs = tflearn.input_data(shape=[None, 3], name="input_1")
net = tflearn.fully_connected(inputs, Layer1)
net = tflearn.layers.normalization.batch_normalization(net)
net = tflearn.activations.relu(net)
net = tflearn.fully_connected(net, Layer2)
net = tflearn.layers.normalization.batch_normalization(net)
net = tflearn.activations.relu(net)
# Final layer weights are init to Uniform[-3e-3, 3e-3]
w_init = tflearn.initializations.uniform(minval=-0.003, maxval=0.003)
out = tflearn.fully_connected(net, 2, activation='sigmoid', weights_init=w_init)
# Scale output to -action_bound to action_bound
scaled_out = tf.multiply(out, 2.0, name="output_1")

network = regression(scaled_out, optimizer='adam', learning_rate=0.0001,
                     loss='mean_square', name='target')

model = tflearn.DNN(network, tensorboard_verbose=0)
# pdb.set_trace()
model.fit({'input_1': X}, {'target': Y}, n_epoch=5,
           validation_set=({'input_1': testX}, {'target': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')


Training Step: 27344  | total loss: 0.45466 | time: 114.029s
| Adam | epoch: 005 | loss: 0.45466 - acc: 0.8288 -- iter: 349952/350000
Training Step: 27345  | total loss: 0.45097 | time: 115.695s
| Adam | epoch: 005 | loss: 0.45097 - acc: 0.8287 | val_loss: 0.46951 - val_acc: 0.8277 -- iter: 350000/350000
--


In [13]:
model.predict(testX[100:110,:]),testY[100:110,:]

(array([[0.41822907, 1.4167991 ],
        [0.4411963 , 1.5315443 ],
        [0.5582641 , 1.5628287 ],
        [0.54022694, 1.5642097 ],
        [0.40826413, 1.4060172 ],
        [0.45484155, 1.3754606 ],
        [0.4523856 , 1.3953626 ],
        [0.3893454 , 1.4250683 ],
        [0.38139427, 1.4162515 ],
        [0.4620319 , 1.5255545 ]], dtype=float32),
 array([[1.15985826, 2.        ],
        [2.        , 2.        ],
        [1.57241406, 2.        ],
        [0.        , 1.        ],
        [0.        , 1.        ],
        [0.        , 0.125     ],
        [0.08705018, 2.        ],
        [0.        , 0.125     ],
        [1.16333002, 2.        ],
        [0.        , 1.        ]]))

In [2]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

# Building convolutional network
network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y}, n_epoch=1,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')


Training Step: 859  | total loss: 0.21477 | time: 65.739s
| Adam | epoch: 001 | loss: 0.21477 - acc: 0.9525 -- iter: 54976/55000
Training Step: 860  | total loss: 0.21633 | time: 68.204s
| Adam | epoch: 001 | loss: 0.21633 - acc: 0.9479 | val_loss: 0.11908 - val_acc: 0.9677 -- iter: 55000/55000
--


In [51]:
Y.shape

(55000, 10)

In [49]:
help(regression)

Help on function regression in module tflearn.layers.estimator:

regression(incoming, placeholder='default', optimizer='adam', loss='categorical_crossentropy', metric='default', learning_rate=0.001, dtype=tf.float32, batch_size=64, shuffle_batches=True, to_one_hot=False, n_classes=None, trainable_vars=None, restore=True, op_name=None, validation_monitors=None, validation_batch_size=None, name=None)
    Regression.
    
    The regression layer is used in TFLearn to apply a regression (linear or
    logistic) to the provided input. It requires to specify a TensorFlow
    gradient descent optimizer 'optimizer' that will minimize the provided
    loss function 'loss' (which calculate the errors). A metric can also be
    provided, to evaluate the model performance.
    
    A 'TrainOp' is generated, holding all information about the optimization
    process. It is added to TensorFlow collection 'tf.GraphKeys.TRAIN_OPS'
    and later used by TFLearn 'models' classes to perform the training

In [45]:
import tensorflow as tf

graph = tf.get_default_graph()
variable_names = [v.name for v in tf.trainable_variables()]
values = model.session.run(variable_names)
print variable_names
# print sess.run('fully_connected_2/weights:0')

[u'Conv2D/W:0', u'Conv2D/b:0', u'Conv2D_1/W:0', u'Conv2D_1/b:0', u'FullyConnected/W:0', u'FullyConnected/b:0', u'FullyConnected_1/W:0', u'FullyConnected_1/b:0', u'FullyConnected_2/W:0', u'FullyConnected_2/b:0']
